<a href="https://colab.research.google.com/github/YuliiaHudz/Data-Analytics-Projects/blob/main/Project_1_A_B_Vision_Data_Driven_Insights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
from google.cloud import bigquery

import pandas as pd

auth.authenticate_user()

client = bigquery.Client(project="data-analytics-mate")

query = """
WITH session_info AS (
    SELECT
        s.date,
        s.ga_session_id,
        sp.country,
        sp.device,
        sp.continent,
        sp.channel,
        ab.test,
        ab.test_group
    FROM `DA.session` s
    JOIN `DA.ab_test` ab USING (ga_session_id)
    JOIN `DA.session_params` sp USING (ga_session_id)
),

aggregated_data AS (
    SELECT
        si.date,
        si.country,
        si.device,
        si.continent,
        si.channel,
        si.test,
        si.test_group,
        COUNT(DISTINCT o.ga_session_id) AS sessions_with_orders,
        COUNT(DISTINCT si.ga_session_id) AS session_cnt,
        COUNT(DISTINCT acs.ga_session_id) AS new_account_cnt
    FROM session_info si
    LEFT JOIN `DA.order` o USING (ga_session_id)
    LEFT JOIN `DA.account_session` acs USING (ga_session_id)
    GROUP BY
        si.date, si.country, si.device, si.continent,
        si.channel, si.test, si.test_group
),

event_counts AS (
    SELECT
        si.date,
        si.country,
        si.device,
        si.continent,
        si.channel,
        si.test,
        si.test_group,
        ep.event_name,
        COUNT(ep.ga_session_id) AS event_cnt
    FROM `DA.event_params` ep
    JOIN session_info si USING (ga_session_id)
    GROUP BY
        si.date, si.country, si.device, si.continent,
        si.channel, si.test, si.test_group, ep.event_name
)

SELECT
    date, country, device, continent, channel, test, test_group,
    'session with orders' AS event_name, sessions_with_orders AS value
FROM aggregated_data
WHERE sessions_with_orders > 0

UNION ALL

SELECT
    date, country, device, continent, channel, test, test_group,
    'session' AS event_name, session_cnt AS value
FROM aggregated_data
WHERE session_cnt > 0

UNION ALL

SELECT
    date, country, device, continent, channel, test, test_group,
    'new account' AS event_name, new_account_cnt AS value
FROM aggregated_data
WHERE new_account_cnt > 0

UNION ALL

SELECT
    date, country, device, continent, channel, test, test_group,
    event_name, event_cnt AS value
FROM event_counts
WHERE event_cnt > 0;
"""
query_job = client.query(query)
results = query_job.result()

df = results.to_dataframe()
df.head()

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Ecuador,desktop,Americas,Organic Search,2,2,session,1
1,2020-11-01,Ecuador,desktop,Americas,Organic Search,1,1,session,1
2,2020-11-01,Uruguay,mobile,Americas,Organic Search,2,2,session,1
3,2020-11-01,Uruguay,mobile,Americas,Organic Search,1,1,session,1
4,2020-11-01,Georgia,mobile,Asia,Direct,2,2,session,1


**Explanation:** In this part, a BigQuery client is set up to interact with Google BigQuery, and a complex SQL query is defined. The query aggregates session and event data, such as sessions with orders, session counts, new accounts, and event counts, grouped by dimensions like date, country, device, and test group. The query is executed in BigQuery, and the results are converted into a Pandas DataFrame for further analysis. Finally, the first few rows of the DataFrame are displayed.

# **Setup and Initialize Variables**

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# Dictionary of metrics for analysis
metrics = {
    "add_payment_info": "add_payment_info / session",
    "add_shipping_info": "add_shipping_info / session",
    "begin_checkout": "begin_checkout / session",
    "new account": "new account / session"
}

# Initialize results list and other variables
test_numbers = df["test"].unique()
results = []

# Aggregate data for faster calculations
df_agg = df.groupby(["test", "test_group", "event_name"])["value"].sum().reset_index()

NameError: name 'df' is not defined

**Explanation:** In this part, the necessary libraries (numpy, pandas, and scipy.stats.norm) are imported. A dictionary, metrics, is created to define formulas for different metrics. The results list is initialized to store the output, and unique test numbers are extracted from the dataset for further analysis. Additionally, the dataset is aggregated to optimize calculations.

# **Process Each Test and Calculate Metrics**

In [ ]:

# Loop through each test in the dataset
for test_value in test_numbers:
    test_subset = df_agg[df_agg["test"] == test_value]

    # Split into control and test groups
    control = test_subset[test_subset["test_group"] == 1].set_index("event_name")["value"]
    test = test_subset[test_subset["test_group"] == 2].set_index("event_name")["value"]

    if "session" not in control or "session" not in test:
        print(f"Missing session data for test '{test_value}'")
        continue

    # Total number of sessions in each group
    denominator_control = control.get("session", 0)
    denominator_experiment = test.get("session", 0)

    if denominator_control == 0 or denominator_experiment == 0:
        print(f"Zero denominator for test '{test_value}', skipping...")
        continue

    # Loop through each metric
    for metric, formula in metrics.items():
        numerator_control = control.get(metric, 0)
        numerator_experiment = test.get(metric, 0)

        conversion_rate_A = numerator_control / denominator_control
        conversion_rate_B = numerator_experiment / denominator_experiment
        metric_change = ((conversion_rate_B - conversion_rate_A) / conversion_rate_A) * 100 if conversion_rate_A > 0 else 0

        n_control = denominator_control
        n_test = denominator_experiment

        pooled_prob = (numerator_control + numerator_experiment) / (n_control + n_test)
        std_error = np.sqrt(pooled_prob * (1 - pooled_prob) * (1 / n_control + 1 / n_test)) if pooled_prob > 0 else 0

        z_stat = (conversion_rate_B - conversion_rate_A) / std_error if std_error > 0 else 0
        p_value = 2 * (1 - norm.cdf(abs(z_stat))) if std_error > 0 else 1

        results.append({
            'test_number': test_value,
            'metric': formula,
            'numerator_event': metric,
            'denominator_event': "session",
            'numerator_converse_A': numerator_control,
            'denominator_converse_A': denominator_control,
            'conversion_rate_A': f"{conversion_rate_A:.10f}",
            'numerator_converse_B': numerator_experiment,
            'denominator_converse_B': denominator_experiment,
            'conversion_rate_B': f"{conversion_rate_B:.10f}",
            'metric_change': f"{metric_change:.10f}",
            'z_stat': f"{z_stat:.10f}",
            'p_value': f"{p_value:.10f}",
            'significant': p_value < 0.05
        })

**Explanation:** This part iterates through each test in the dataset and splits the data into control and test groups. If either group is missing data, the test is skipped. For each metric, the sum of values is calculated for both groups, and cases where denominators are zero are handled. Conversion rates are computed, followed by the metric change. The pooled probability, standard error, z-statistic, and p-value are also determined. Finally, results are stored in the results list, including details such as conversion rates, metric change, statistical significance, and the z-statistic.

# **Convert Results**

In [ ]:
# Convert results to DataFrame and sort
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="test_number", ascending=True)

# Save to CSV and Excel
results_df.to_csv("ab_test_results.csv", index=False)
results_df.to_excel("ab_test_results.xlsx", index=False)

print("Results saved to 'ab_test_results.csv' and 'ab_test_results.xlsx'.")


Results saved to 'ab_test_results.csv' and 'ab_test_results.xlsx'.


# **Save Results to CSV and Excel Files**

In [ ]:
from google.colab import files

# Download the CSV and Excel files directly
files.download("ab_test_results.csv")
files.download("ab_test_results.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Results**

[**Stage 1. Statistical Significance Calculations**](https://docs.google.com/spreadsheets/d/1uJ3ILjfhe2S_1PY0e2XGkpAYkaCItS-i/edit?usp=sharing&ouid=112933793969429000884&rtpof=true&sd=true)


[**Stage 2. Visualization in Tableau**](https://public.tableau.com/app/profile/yuliia.hudz/viz/ABVisionData-DrivenInsights/ABtest)